<a href="https://colab.research.google.com/github/UzunDemir/9_matrix_knn/blob/main/9_6_practic_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.6 Практическая работа
## Цель практической работы

Научиться:

* применять метод матричного разложения для задачи рекомендательных систем;
* раскладывать матрицу оценок, используя метод матричного разложения;
* рассчитывать метрику mAP.


## Что входит в работу
* Создать user-item-матрицу и разбить данные на тест и контроль результатов.
* Создать бейзлайны и рассчитать метрики.
* Применить метод матричной факторизации и улучшить параметры алгоритма факторизации.


## Задание 1. Создание user-item-матрицы и разбиение данных на тест и контроль результатов
### Что нужно сделать

Постройте user-item-матрицу на основе данных о прочитанных книгах из таблицы [ratings.csv](https://drive.google.com/file/d/1BLR_z7h2OQ6ISUVLjVApWLeqRR8EDUAS/view).

1. Подключите библиотеку Implicit. Для лучшей работы с библиотекой переведите данные рейтинга в бинарные оценки: 1 — книга понравилась, 0 — книга не понравилась. Необходимо предсказать, понравится ли конкретная книга заданному пользователю, то есть высчитать implicit-оценки.
2. Для дальнейшей оценки качества разбейте данные на тест и контроль. Так как в данных нет настоящей даты прочтения, воспользуйтесь обходным способом:
3. Пронумеруйте для каждого пользователя его прочитанные книги согласно расположению книг по порядку.
4. Переведите номера прочитанных книг в доли по формуле «порядковый номер / общее количество прочитанных пользователем книг».
5. Определите, какое количество данных вы оставите на обучение, какое — на контроль. Например, 70% книг каждого пользователя — на обучение.
6. На основе данных для обучения постройте user-item-матрицу.
7. Посмотрите, сколько в матрице столбцов и строк. Совпадает ли их количество с числом уникальных пользователей и книг?
8. Выведите список названий книг с высокой оценкой для любого из пользователей.
9. Запомните индекс пользователя, он пригодится далее.

In [1]:
import gdown

# URL вашего файла
url = "https://drive.google.com/uc?id=1BLR_z7h2OQ6ISUVLjVApWLeqRR8EDUAS"

# Путь для сохранения файла
output = "ratings.csv"

gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BLR_z7h2OQ6ISUVLjVApWLeqRR8EDUAS
To: /content/ratings.csv
100%|██████████| 72.1M/72.1M [00:04<00:00, 16.0MB/s]


'ratings.csv'

In [3]:
import pandas as pd

# Открываем файл в DataFrame
df = pd.read_csv('/content/ratings.csv')

# Выводим первые несколько строк DataFrame для проверки
df.head()


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
df.shape

(5976479, 3)

In [5]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB
None


In [6]:
print(df['user_id'].nunique())  # количество уникальных пользователей
print(df['book_id'].nunique())  # количество уникальных книг
print(df['rating'].unique())    # уникальные рейтинги


53424
10000
[5 4 3 2 1]


In [7]:
average_ratings = df.groupby('book_id')['rating'].mean()
print(average_ratings)


book_id
1        4.279707
2        4.351350
3        3.214341
4        4.329369
5        3.772224
           ...   
9996     4.014184
9997     4.451613
9998     4.323529
9999     3.707692
10000    4.009009
Name: rating, Length: 10000, dtype: float64


In [8]:
most_rated_book = df['book_id'].value_counts().idxmax()
print(f"Book ID with the most ratings: {most_rated_book}")


Book ID with the most ratings: 1


In [9]:
# Считаем количество оценок для каждой книги
ratings_count = df['book_id'].value_counts()

# Выводим первые 100 книг с наибольшим количеством оценок
top_100_books = ratings_count.head(100)
print(top_100_books)


book_id
1      22806
2      21850
4      19088
3      16931
5      16604
       ...  
102     6401
100     6343
161     6331
73      6313
129     6307
Name: count, Length: 100, dtype: int64


In [9]:
!pip install implicit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.1 MB/s eta 0:00:00


In [10]:
import pandas as pd
import implicit
from scipy.sparse import csr_matrix
import numpy as np

In [11]:
# Преобразование рейтингов в бинарные оценки: 1 — понравилось, 0 — не понравилось
df['binary_rating'] = df['rating'].apply(lambda x: 1 if x >= 3 else 0)

# Вывод первых строк для проверки
print(df.head())

   user_id  book_id  rating  binary_rating
0        1      258       5              1
1        2     4081       4              1
2        2      260       5              1
3        2     9296       5              1
4        2     2318       3              1


In [12]:
df.binary_rating.value_counts()

,count
binary_rating,
1,5493027
0,483452


In [13]:
# Создание разреженной матрицы user-item
user_item_matrix = csr_matrix((df['binary_rating'], (df['user_id'], df['book_id'])))

# Вывод размера матрицы для проверки
print(f"Размер матрицы: {user_item_matrix.shape}")


Размер матрицы: (53425, 10001)


In [14]:
# Создание модели Alternating Least Squares (ALS)
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Обучение модели на матрице user-item
model.fit(user_item_matrix.T)


/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.1281125545501709 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
# Нумерация книг для каждого пользователя
df['rank'] = df.groupby('user_id').cumcount() + 1

# Преобразование рангов в доли
df['rank_proportion'] = df['rank'] / df.groupby('user_id')['rank'].transform('max')

# Вывод первых строк для проверки
print(df.head())


   user_id  book_id  rating  binary_rating  rank  rank_proportion
0        1      258       5              1     1         0.008547
1        2     4081       4              1     1         0.015385
2        2      260       5              1     2         0.030769
3        2     9296       5              1     3         0.046154
4        2     2318       3              1     4         0.061538


Здесь мы устанавливаем, что 70% данных каждого пользователя будет использоваться для обучения, а оставшиеся 30% — для контроля.

In [16]:
# Установите процент данных для обучения и контроля
train_proportion = 0.7

# Определим, какие записи оставить для обучения
df['is_train'] = df['rank_proportion'] <= train_proportion


In [17]:
# Создание разреженной матрицы user-item для обучающих данных
train_matrix = csr_matrix((df[df['is_train']]['binary_rating'],
                           (df[df['is_train']]['user_id'], df[df['is_train']]['book_id'])))

# Вывод размера матрицы для проверки
print(f"Размер обучающей матрицы: {train_matrix.shape}")


Размер обучающей матрицы: (53425, 10001)


In [18]:
print(df['user_id'].max(), df['book_id'].max())


53424 10000


In [19]:
print(df['user_id'].nunique(), df['book_id'].nunique())


53424 10000


In [20]:
print(df[df['user_id'] < 0])
print(df[df['book_id'] < 0])
print(df[df['user_id'].isna()])
print(df[df['book_id'].isna()])


Empty DataFrame
Columns: [user_id, book_id, rating, binary_rating, rank, rank_proportion, is_train]
Index: []
Empty DataFrame
Columns: [user_id, book_id, rating, binary_rating, rank, rank_proportion, is_train]
Index: []
Empty DataFrame
Columns: [user_id, book_id, rating, binary_rating, rank, rank_proportion, is_train]
Index: []
Empty DataFrame
Columns: [user_id, book_id, rating, binary_rating, rank, rank_proportion, is_train]
Index: []


In [21]:
print(df[df.duplicated(['user_id', 'book_id'], keep=False)])


Empty DataFrame
Columns: [user_id, book_id, rating, binary_rating, rank, rank_proportion, is_train]
Index: []


In [22]:
print(df.dtypes)


user_id              int64
book_id              int64
rating               int64
binary_rating        int64
rank                 int64
rank_proportion    float64
is_train              bool
dtype: object


In [23]:
user_id = 3  # идентификатор пользователя
user_ratings = train_matrix[user_id].toarray().flatten()
high_rated_books = np.where(user_ratings == 1)[0]

print(f"Книги с высокой оценкой для пользователя {user_id}: {high_rated_books}")


Книги с высокой оценкой для пользователя 3: [   4   59  118  247  410  715  977 1714 2963 3311 7401 8492]


## Задание 2. Создание бейзлайнов и расчёт метрик
### Что нужно сделать
Для оценки качества модели создайте несколько простых бейзлайнов.

1. Реализуйте функцию, которая подсчитывает метрику AP@K. На вход функция принимает список рекомендаций и список книг, положительно оценённых пользователем (с оценкой 4 или 5).
2. Зафиксируйте случайный список из 500 пользовательских индексов. Подсчитайте случайный бейзлайн — для каждого пользователя из списка сгенерируйте случайный ответ из 20 книг. Удалите книги, которые пользователь уже читал.
3. Рассчитайте метрику AP@10 для каждого из пользователей и усредните её по всем пользователям.
4. Аналогично постройте бейзлайн из самых популярных книг: посмотрите, какие книги пользователи читали больше всего, и сделайте рекомендации на их основе. Подсчитайте mAP@10.

### 1. Реализация функции AP@K
Метрика AP@K (Average Precision at K) измеряет, насколько хорошо рекомендации соответствуют интересам пользователя. Она вычисляется как среднее значение прецизионности в топ-K рекомендациях.

In [24]:
def ap_at_k(recommendations, relevant_items, k=10):
    """
    Calculate Average Precision at K (AP@K)

    :param recommendations: List of recommended item IDs
    :param relevant_items: List of relevant item IDs
    :param k: Number of top recommendations to consider
    :return: AP@K score
    """
    recommendations = recommendations[:k]
    relevant_items_set = set(relevant_items)
    num_relevant = 0
    precision_sum = 0.0

    for i, item in enumerate(recommendations):
        if item in relevant_items_set:
            num_relevant += 1
            precision_sum += num_relevant / (i + 1)

    if num_relevant == 0:
        return 0.0

    return precision_sum / min(len(relevant_items), k)


* recommendations: список рекомендованных книг.
* relevant_items: список книг, которые пользователь оценил положительно.
* k: число рекомендованных книг для оценки (обычно 10).

Функция подсчитывает, насколько высока прецизионность рекомендованных книг в топ-K.

In [25]:
import random

# Фиксация случайного списка из 500 пользовательских индексов
random_users = df['user_id'].sample(n=500, random_state=42).unique()

def generate_random_recommendations(user_id, all_books, read_books, num_recommendations=20):
    """
    Generate random recommendations for a user

    :param user_id: User ID
    :param all_books: List of all book IDs
    :param read_books: List of books that the user has already read
    :param num_recommendations: Number of recommendations to generate
    :return: List of recommended book IDs
    """
    unread_books = list(set(all_books) - set(read_books))
    return random.sample(unread_books, min(num_recommendations, len(unread_books)))

# Подсчет случайного бейзлайна для каждого пользователя
def calculate_random_baseline(user_ids, all_books, ratings_df):
    """
    Calculate random baseline for given users

    :param user_ids: List of user IDs
    :param all_books: List of all book IDs
    :param ratings_df: DataFrame containing ratings
    :return: Dictionary with user IDs and their AP@10 scores
    """
    ap_scores = {}

    for user_id in user_ids:
        read_books = ratings_df[ratings_df['user_id'] == user_id]['book_id'].tolist()
        recommendations = generate_random_recommendations(user_id, all_books, read_books)
        relevant_books = ratings_df[ratings_df['user_id'] == user_id]['book_id'][ratings_df['rating'] >= 4].tolist()
        ap_score = ap_at_k(recommendations, relevant_books, k=10)
        ap_scores[user_id] = ap_score

    return ap_scores

# Генерация списка всех книг
all_books = df['book_id'].unique()

# Подсчет AP@10 для случайного бейзлайна
random_baseline_scores = calculate_random_baseline(random_users, all_books, df)
mean_ap_random = np.mean(list(random_baseline_scores.values()))
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random}")


Средний AP@10 для случайного бейзлайна: 0.0


In [26]:
# Подсчет числа прочтений каждой книги
book_popularity = df['book_id'].value_counts()

# Выбираем 20 самых популярных книг
top_books = book_popularity.head(20).index.tolist()

def calculate_popular_books_baseline(user_ids, top_books, ratings_df):
    """
    Calculate popular books baseline for given users

    :param user_ids: List of user IDs
    :param top_books: List of popular book IDs
    :param ratings_df: DataFrame containing ratings
    :return: Dictionary with user IDs and their AP@10 scores
    """
    ap_scores = {}

    for user_id in user_ids:
        read_books = ratings_df[ratings_df['user_id'] == user_id]['book_id'].tolist()
        recommendations = [book for book in top_books if book not in read_books]
        relevant_books = ratings_df[ratings_df['user_id'] == user_id]['book_id'][ratings_df['rating'] >= 4].tolist()
        ap_score = ap_at_k(recommendations, relevant_books, k=10)
        ap_scores[user_id] = ap_score

    return ap_scores

# Подсчет AP@10 для бейзлайна из самых популярных книг
popular_baseline_scores = calculate_popular_books_baseline(random_users, top_books, df)
mean_ap_popular = np.mean(list(popular_baseline_scores.values()))
print(f"Средний AP@10 для бейзлайна из популярных книг: {mean_ap_popular}")


Средний AP@10 для бейзлайна из популярных книг: 0.0


In [27]:
def generate_random_recommendations(user_id, all_books, read_books, num_recommendations=20):
    """
    Generate random recommendations for a user

    :param user_id: User ID
    :param all_books: List of all book IDs
    :param read_books: List of books that the user has already read
    :param num_recommendations: Number of recommendations to generate
    :return: List of recommended book IDs
    """
    unread_books = list(set(all_books) - set(read_books))
    if len(unread_books) < num_recommendations:
        return unread_books
    return random.sample(unread_books, num_recommendations)

# Проверим случайные рекомендации для одного пользователя
user_id = random_users[0]
read_books = df[df['user_id'] == user_id]['book_id'].tolist()
recommendations = generate_random_recommendations(user_id, all_books, read_books)
print(f"Recommendations for user {user_id}: {recommendations}")


Recommendations for user 42562: [7902, 3356, 3475, 5771, 1771, 4266, 2473, 3982, 941, 5881, 1008, 2409, 3120, 1245, 5857, 9676, 1573, 6002, 3758, 1003]


In [28]:
def ap_at_k(recommendations, relevant_items, k=10):
    """
    Calculate Average Precision at K (AP@K)

    :param recommendations: List of recommended item IDs
    :param relevant_items: List of relevant item IDs
    :param k: Number of top recommendations to consider
    :return: AP@K score
    """
    recommendations = recommendations[:k]
    relevant_items_set = set(relevant_items)
    num_relevant = 0
    precision_sum = 0.0

    for i, item in enumerate(recommendations):
        if item in relevant_items_set:
            num_relevant += 1
            precision_sum += num_relevant / (i + 1)

    if num_relevant == 0:
        return 0.0

    return precision_sum / min(len(relevant_items), k)

# Проверим AP@K для простого случая
recommendations = [1, 2, 3, 4, 5]
relevant_items = [1, 2, 6, 7, 8]
print(f"AP@10: {ap_at_k(recommendations, relevant_items, k=10)}")


AP@10: 0.4


In [29]:
# Проверим, есть ли у пользователя релевантные книги
user_id = random_users[0]
relevant_books = df[(df['user_id'] == user_id) & (df['rating'] >= 4)]['book_id'].tolist()
print(f"Relevant books for user {user_id}: {relevant_books}")


Relevant books for user 42562: [144, 179, 767, 465, 38, 46, 150, 417, 1361, 195, 291, 613, 482, 1256, 31, 22, 1237, 311, 3143, 1758, 100, 16, 4854, 115, 516, 1, 33, 925, 1458, 19, 94, 669, 457, 212, 1133, 453, 8435, 109, 1073, 3, 72, 143]


In [30]:
print(f"Top books: {top_books}")


Top books: [1, 2, 4, 3, 5, 17, 20, 18, 23, 7, 24, 25, 21, 27, 13, 8, 16, 14, 28, 9]


In [31]:
def ap_at_k(recommendations, relevant_items, k=10):
    """
    Calculate Average Precision at K (AP@K).

    :param recommendations: List of recommended item IDs (top-K recommendations).
    :param relevant_items: List of relevant item IDs (items positively rated by the user).
    :param k: Number of top recommendations to consider.
    :return: AP@K score.
    """
    # Slice recommendations to consider only top-K recommendations
    recommendations = recommendations[:k]

    # Convert relevant_items to a set for faster lookups
    relevant_items_set = set(relevant_items)

    # Initialize variables
    num_relevant = 0
    precision_sum = 0.0

    # Iterate over the recommendations to calculate precision
    for i, item in enumerate(recommendations):
        if item in relevant_items_set:
            num_relevant += 1
            precision_sum += num_relevant / (i + 1)

    # Return AP@K score, handling the case where there are no relevant items
    if num_relevant == 0:
        return 0.0

    return precision_sum / min(len(relevant_items), k)

# Пример использования
recommendations = [1, 2, 4, 3, 5, 17, 20, 18, 23, 7, 24, 25, 21, 27, 13, 8, 16, 14, 28, 9]
relevant_items = [2, 4, 6, 8, 10]
print(f"AP@10: {ap_at_k(recommendations, relevant_items, k=10)}")


AP@10: 0.2333333333333333


In [32]:
def get_recommendations(user_id, model, all_books, num_recommendations=20):
    """
    Получите список рекомендаций для пользователя на основе модели.

    :param user_id: ID пользователя
    :param model: Рекомендательная модель
    :param all_books: Список всех книг
    :param num_recommendations: Количество рекомендаций
    :return: Список рекомендованных книг
    """
    # Предсказание оценок для всех книг
    user_vector = np.array([user_id])  # Преобразование user_id в нужный формат
    predictions = model.predict(user_vector, np.arange(len(all_books)))

    # Создание DataFrame для удобного просмотра и сортировки
    recommendations_df = pd.DataFrame({
        'book_id': all_books,
        'predicted_rating': predictions.flatten()
    })

    # Сортировка книг по предсказанным оценкам
    recommendations_df = recommendations_df.sort_values(by='predicted_rating', ascending=False)

    # Выбор топ-K рекомендаций
    top_recommendations = recommendations_df.head(num_recommendations)['book_id'].tolist()

    return top_recommendations


In [33]:
def get_positive_books(user_id, ratings_df):
    """
    Получите список книг, положительно оценённых пользователем.

    :param user_id: ID пользователя
    :param ratings_df: DataFrame с данными о рейтингах
    :return: Список книг с положительными оценками
    """
    # Фильтрация данных для данного пользователя и оценок 4 или 5
    positive_books = ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['rating'] >= 4)]

    return positive_books['book_id'].tolist()


In [34]:
from implicit.als import AlternatingLeastSquares
# Загрузка данных
ratings_df = pd.read_csv('ratings.csv')

# Создание user-item-матрицы
user_item_matrix = csr_matrix((ratings_df['rating'], (ratings_df['user_id'], ratings_df['book_id'])))

# Инициализация и обучение модели AlternatingLeastSquares
model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(user_item_matrix)


  0%|          | 0/20 [00:00<?, ?it/s]

In [35]:
def get_recommendations(model, user_id, user_item_matrix, num_recommendations=10):
    """
    Получить рекомендации для заданного пользователя.

    :param model: Обученная модель ALS
    :param user_id: ID пользователя
    :param user_item_matrix: Матрица user-item
    :param num_recommendations: Количество рекомендаций
    :return: Список рекомендованных книг
    """
    # Получить все книги
    all_books = user_item_matrix.shape[1]

    # Получить оценки для данного пользователя
    user_ratings = user_item_matrix[user_id]

    # Получить рекомендации
    recommendations = model.recommend(user_id, user_item_matrix[user_id], N=num_recommendations)

    # Возвращаем книги и их оценку
    return recommendations




In [36]:
# Пример использования
user_id = 3  # ID пользователя
recommendations = get_recommendations(model, user_id, user_item_matrix)
print(recommendations)

(array([ 14,  29,   5,   8, 125,  87, 154, 131,  48, 354], dtype=int32), array([0.52084905, 0.5057618 , 0.50478673, 0.49874854, 0.44828695,
       0.44163695, 0.43954957, 0.42903876, 0.416268  , 0.40675738],
      dtype=float32))


### Описание метрики:
* Precision at K (P@K): Точность на уровне K — это доля правильно рекомендованных книг среди первых K книг в списке рекомендаций.

* Average Precision (AP@K): Это среднее значение Precision на уровнях от 1 до K. Важно учитывать порядок, так как более высокие позиции в списке рекомендаций имеют больший вес.

In [37]:
def average_precision_at_k(recommendations, relevant_books, k):
    """
    Вычисляет Average Precision at K (AP@K).

    :param recommendations: Список рекомендованных книг (отсортированный по убыванию вероятности)
    :param relevant_books: Список книг, положительно оцененных пользователем (с оценкой 4 или 5)
    :param k: Количество верхних рекомендаций, для которых рассчитывается метрика
    :return: Average Precision at K
    """
    # Обрезаем список рекомендаций до K
    recommendations = recommendations[:k]

    # Определяем количество релевантных книг среди первых K рекомендаций
    relevant_count = 0
    precision_sum = 0.0

    for i, book in enumerate(recommendations):
        if book in relevant_books:
            relevant_count += 1
            precision_sum += relevant_count / (i + 1)  # i + 1 потому что индексы начинаются с 0

    # AP@K вычисляется как среднее значение Precision по всем позициям, где обнаружены релевантные книги
    if relevant_count > 0:
        ap_at_k = precision_sum / min(k, len(relevant_books))
    else:
        ap_at_k = 0.0

    return ap_at_k


In [40]:
# Список книг, рекомендованных пользователю (отсортированный по вероятности)
recommendations = [14,   5,  29,   8,  87, 125, 154,  48, 131,  10]

# Список книг, положительно оцененных пользователем (с оценкой 4 или 5)
relevant_books = [172, 195, 118, 5]

# Количество рекомендаций для оценки
k = 10

# Вычисляем AP@K
ap_at_k = average_precision_at_k(recommendations, relevant_books, k)
print(f"AP@{k}: {ap_at_k}")


AP@10: 0.125


* Обрезка до K: Мы берем только первые K рекомендаций, так как нас интересуют только верхние рекомендации.
* Расчет Precision: Для каждой книги в первых K рекомендациях проверяем, является ли она релевантной. Если да, то добавляем её Precision к сумме Precision.
* AP@K: Делим сумму Precision на количество релевантных книг, чтобы получить среднее значение Precision на уровне K.

In [41]:
# Получение уникальных идентификаторов пользователей
user_ids = ratings_df['user_id'].unique()

# Выбор случайных 500 пользователей
np.random.seed(0)  # Устанавливаем seed для воспроизводимости
random_users = np.random.choice(user_ids, 500, replace=False)

Для каждого выбранного пользователя создадим случайный список из 20 книг, исключая книги, которые он уже читал.

In [42]:
from collections import defaultdict

# Словарь для хранения книг, которые уже читал каждый пользователь
user_books = defaultdict(set)
for _, row in ratings_df.iterrows():
    user_books[row['user_id']].add(row['book_id'])

# Генерация случайных рекомендаций
books = ratings_df['book_id'].unique()  # Все уникальные книги

def generate_random_recommendations(user_id, num_recommendations=20):
    # Книги, которые пользователь уже читал
    read_books = user_books[user_id]

    # Фильтрация книг, которые пользователь уже читал
    possible_books = [book for book in books if book not in read_books]

    # Генерация случайного списка книг
    if len(possible_books) > num_recommendations:
        recommendations = np.random.choice(possible_books, num_recommendations, replace=False)
    else:
        recommendations = possible_books

    return recommendations

# Пример для одного случайного пользователя
user_id = random_users[0]
recommendations = generate_random_recommendations(user_id)
print(f"Случайные рекомендации для пользователя {user_id}: {recommendations}")


Случайные рекомендации для пользователя 4722: [6971 4682 6501  425 1308  682 3315   33 9825 6483 7111  248 3868 4309
 4594 5823 9146 3444 2411 8226]


Теперь вычислим метрику AP@10 для случайных рекомендаций. Нужно вычислить метрику для каждого пользователя и усреднить по всем пользователям.

In [43]:
def average_precision_at_k(recommendations, relevant_books, k):
    """
    Вычисляет Average Precision at K (AP@K).
    """
    recommendations = recommendations[:k]
    relevant_count = 0
    precision_sum = 0.0

    for i, book in enumerate(recommendations):
        if book in relevant_books:
            relevant_count += 1
            precision_sum += relevant_count / (i + 1)

    if relevant_count > 0:
        ap_at_k = precision_sum / min(k, len(relevant_books))
    else:
        ap_at_k = 0.0

    return ap_at_k

# Список для хранения метрик AP@10
ap_at_10_list = []

# Вычисляем AP@10 для каждого пользователя из списка
for user_id in random_users:
    # Книги, положительно оцененные пользователем
    relevant_books = ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['rating'] >= 4)]['book_id'].tolist()

    # Генерация случайных рекомендаций
    recommendations = generate_random_recommendations(user_id)

    # Вычисляем AP@10
    ap_at_10 = average_precision_at_k(recommendations, relevant_books, 10)
    ap_at_10_list.append(ap_at_10)

# Усреднение AP@10
average_ap_at_10 = np.mean(ap_at_10_list)
print(f"Средний AP@10 для случайного бейзлайна: {average_ap_at_10}")


Средний AP@10 для случайного бейзлайна: 0.0


In [44]:
# Определим популярные книги
popular_books = ratings_df.groupby('book_id').size().sort_values(ascending=False).index


In [45]:
# Выберем топ-20 популярных книг
top_n_books = popular_books[:20]


In [46]:
print(f"Топ-20 популярных книг: {top_n_books}")


Топ-20 популярных книг: Index([1, 2, 4, 3, 5, 17, 20, 18, 23, 7, 24, 25, 21, 27, 13, 8, 16, 14, 28, 9], dtype='int64', name='book_id')


In [47]:
import numpy as np

def get_popular_recommendations(user_id, top_n_books, user_item_matrix):
    # Получим книги, которые пользователь уже читал
    user_books = set(user_item_matrix[user_id, :].nonzero()[1])

    # Создадим список рекомендаций из самых популярных книг
    recommendations = [book for book in top_n_books if book not in user_books]

    return recommendations[:20]

# Генерируем рекомендации для случайного списка пользователей
random_users = np.random.choice(user_item_matrix.shape[0], 500, replace=False)
popular_recommendations = {}

for user in random_users:
    popular_recommendations[user] = get_popular_recommendations(user, top_n_books, user_item_matrix)


In [48]:
def get_positive_books(user_id, ratings_df):
    return set(ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['rating'] >= 4)]['book_id'])

positive_books = {user: get_positive_books(user, ratings_df) for user in random_users}


In [49]:
def mean_average_precision_at_k(recommendations, positive_books, k=10):
    ap_scores = []

    for user, recs in recommendations.items():
        ap_scores.append(average_precision_at_k(recs, positive_books.get(user, set()), k))

    return np.mean(ap_scores)

# Рассчитайте среднее значение AP@10 для бейзлайна из популярных книг
mAP_at_10_popular = mean_average_precision_at_k(popular_recommendations, positive_books, k=10)
print(f"Средний AP@10 для бейзлайна из популярных книг: {mAP_at_10_popular:.4f}")


Средний AP@10 для бейзлайна из популярных книг: 0.0000


In [50]:
# Пример использования
user_id = 2  # ID пользователя
recommendations = get_recommendations(model, user_id, user_item_matrix)
print(recommendations)

(array([ 92,  14,  13,   4,  35, 247,  28,  32,  55,  42], dtype=int32), array([0.904475  , 0.8670933 , 0.8527905 , 0.8398323 , 0.7991242 ,
       0.7467043 , 0.74256724, 0.69936424, 0.597841  , 0.5927464 ],
      dtype=float32))


## Задание 3. Применение метода матричной факторизации и улучшение параметров алгоритма факторизации

### Что нужно сделать
1. Используя результаты, полученные при выполнении заданий 1 и 2, обучите алгоритм ALS из библиотеки Implicit с базовыми параметрами. Посмотрите, какие рекомендации вы получите для конкретного пользователя.
2. Используя полученный ранее случайный список пользователей из задания 2, подсчитайте метрику mAP@10. Превзошла ли она какой-нибудь из бейзлайнов?
3. Попробуйте использовать более «продвинутые» параметры:
* большее количество факторов — отвечает за размерность векторов;
* больше итераций, чтобы приблизиться к ответу.
4. Сделайте несколько версий и посмотрите, как будут меняться рекомендации для конкретного пользователя и общая метрика mAP@10 (вы должны превзойти значение 0,02 — это нормально). Сделайте выводы о качестве полученной модели.

In [107]:
# Создание модели Alternating Least Squares (ALS)
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Обучение модели на матрице user-item
model.fit(user_item_matrix.T)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.15095162391662598 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [51]:
# Пример использования
user_id = 3  # ID пользователя
recommendations = get_recommendations(model, user_id, user_item_matrix)
print(recommendations)

(array([ 14,  29,   5,   8, 125,  87, 154, 131,  48, 354], dtype=int32), array([0.52084905, 0.5057618 , 0.50478673, 0.49874854, 0.44828695,
       0.44163695, 0.43954957, 0.42903876, 0.416268  , 0.40675738],
      dtype=float32))


In [52]:
user_id = 3  # идентификатор пользователя
user_ratings = train_matrix[user_id].toarray().flatten()
high_rated_books = np.where(user_ratings == 1)[0]

print(f"Книги с высокой оценкой для пользователя {user_id}: {high_rated_books}")


Книги с высокой оценкой для пользователя 3: [   4   59  118  247  410  715  977 1714 2963 3311 7401 8492]


In [ ]:
# 14,  29,   5,   8, 125,  87, 154, 131,  48, 354

In [55]:

recommendations = [14,  29,   5,   8, 125,  87, 154, 131,  48, 354]

# Список книг, положительно оцененных пользователем (с оценкой 4 или 5)
relevant_books = [172, 195, 118, 5]

# Количество рекомендаций для оценки
k = 10

# Вычисляем AP@K
ap_at_k = average_precision_at_k(recommendations, relevant_books, k)
print(f"AP@{k}: {ap_at_k}")

AP@10: 0.08333333333333333


Оптимизация параметров ALS:

* factors (количество факторов): определяет размерность латентного пространства. Это количество скрытых признаков, которые описывают пользователей и книги. Увеличение этого числа может улучшить качество рекомендаций, но также увеличивает сложность модели и время обучения.
* regularization (регуляризация): контролирует степень регуляризации, чтобы избежать переобучения. Более высокие значения могут улучшить обобщающую способность модели, но слишком высокая регуляризация может ухудшить точность.
* iterations (количество итераций): определяет, сколько раз модель будет обновлять матрицы пользователей и элементов. Увеличение числа итераций может повысить точность модели, но также увеличит время обучения.

In [64]:
# Создание модели Alternating Least Squares (ALS)
model = implicit.als.AlternatingLeastSquares(factors=60, regularization=0.1, iterations=50)

# Обучение модели на матрице user-item
model.fit(user_item_matrix.T)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.15874099731445312 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [78]:
user_id = 3  # ID пользователя
recommendations = get_recommendations(model, user_id, user_item_matrix)
print(recommendations)

(array([28147, 36526, 28437, 35529,  9562, 37710,  8995, 14889, 13108,
       20405], dtype=int32), array([1.5504405, 1.5368575, 1.5239139, 1.5199897, 1.5177922, 1.5001006,
       1.4949894, 1.4793109, 1.4637859, 1.4566276], dtype=float32))


In [73]:
recommendations = [28147, 36526, 28437, 35529,  9562, 37710,  8995, 14889, 13108,
       20405]

# Список книг, положительно оцененных пользователем (с оценкой 4 или 5)
relevant_books = [247, 36, 28437, 359,  9562, 310,  8995, 149, 138,
       20405]

# Количество рекомендаций для оценки
k = 10

# Вычисляем AP@K
ap_at_k = average_precision_at_k(recommendations, relevant_books, k)
print(f"AP@{k}: {ap_at_k}")

AP@10: 0.1561904761904762


In [75]:
import numpy as np

def average_precision_at_k(recommended_books, high_rated_books, K):
    """
    Рассчитывает Average Precision@K.

    Parameters:
    - recommended_books: Список рекомендованных книг.
    - high_rated_books: Список книг, которые пользователь оценил высоко.
    - K: Количество рекомендаций.

    Returns:
    - average_precision: Average Precision@K.
    """
    recommended_books = recommended_books[:K]
    relevant_count = 0
    precision_sum = 0.0

    for i, book_id in enumerate(recommended_books):
        if book_id in high_rated_books:
            relevant_count += 1
            precision_sum += relevant_count / (i + 1)

    return precision_sum / min(K, len(high_rated_books)) if high_rated_books else 0

def mean_average_precision_at_k(user_recommendations, user_high_rated_books, K):
    """
    Рассчитывает Mean Average Precision@K по всем пользователям.

    Parameters:
    - user_recommendations: Словарь, где ключ - ID пользователя, значение - список рекомендованных книг.
    - user_high_rated_books: Словарь, где ключ - ID пользователя, значение - список книг с высокой оценкой.
    - K: Количество рекомендаций.

    Returns:
    - mean_average_precision: Mean Average Precision@K.
    """
    ap_list = []
    for user_id in user_recommendations.keys():
        recommended_books = user_recommendations[user_id]
        high_rated_books = user_high_rated_books[user_id]
        ap = average_precision_at_k(recommended_books, high_rated_books, K)
        ap_list.append(ap)

    return np.mean(ap_list)

# Пример использования
user_recommendations = {
    1: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    2: [150, 250, 350, 450, 550, 650, 750, 850, 950, 1050],
    # Добавьте рекомендации для всех пользователей
}

user_high_rated_books = {
    1: [100, 300, 500],
    2: [150, 350, 550],
    # Добавьте высоко оцененные книги для всех пользователей
}

K = 10
mean_ap_at_k = mean_average_precision_at_k(user_recommendations, user_high_rated_books, K)
print(f"mAP@{K}: {mean_ap_at_k:.4f}")


mAP@10: 0.7556
